# Кластеризация новостей

Для представления текста в виде вектора используется tf-idf с n-gram, для нормализации слов используется Yandex MyStem, 

# Модель новости

In [1]:
import json

class News:
    def __init__(self, id, date, annotation, title, content, url):
        self.id = id
        self.date = date
        self.annotation = annotation
        self.title = title
        self.content = content
        self.url = url
    
    @classmethod
    def from_json(cls, json_str):
        json_dict = json.loads(json_str)
        return cls(**json_dict)

# Загружаем данные

In [2]:
news = []
with open('/data/kasandra/news-1000.json', encoding="utf8") as f:
    for line in f:
        news.append(News.from_json(line))

stopwords = []

with open('../main/python/res/stopwords.txt', mode="r", encoding="utf8") as file:
    for line in file:
        stopwords.append(line.replace('\n', ''))

# Подключаем Yandex MyStem и получаем tfidf vector

In [113]:
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np
from itertools import groupby

mystem = Mystem(mystem_bin='/data/mystem/mystem',
                entire_input=False)

# Оставляем только русские слова
r = re.compile('^[А-ЯЙа-яй]*$')

def lemmatize_rus(text):
    return list(filter(r.match, mystem.lemmatize(text)))

def lemmatize_full(text):
    return mystem.lemmatize(text)

def assign_label_to_news(tuplezz):
    (nws, lbl) = tuplezz
    nws.label = lbl.item()
    return nws

def zip_news(n,l):
    return list(map(assign_label_to_news, zip(n, l)))

tfidf_vectorizer = TfidfVectorizer(use_idf=True,
                                   #max_df=2, min_df=1,
                                   tokenizer=lemmatize_full,
                                   stop_words=stopwords,
                                   ngram_range=(1,2),
                                   decode_error = "ignore")

tfidf_matrix = tfidf_vectorizer.fit_transform(map(lambda n: n.content, news))
tfidf_vectorizer.get_feature_names()[:10]

['a',
 'a bad',
 'a bar',
 'a beautiful',
 'a d',
 'a depiction',
 'a fire',
 'a funny',
 'a gif',
 'a good']

# DBSCAN

In [119]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

X = (tfidf_matrix * tfidf_matrix.T).A
#X2 = StandardScaler(with_mean=False).fit_transform(tfidf_matrix)

db = DBSCAN(eps=1.2, min_samples=2).fit(tfidf_matrix)

print('count clusters: %d' % (len(set(db.labels_)) - (1 if -1 in db.labels_ else 0)))

labels = db.labels_
labels[:100]

count clusters: 133


array([-1,  0, -1, -1, -1,  1,  2, -1, -1,  3,  4, -1, -1, -1, -1,  5, -1,
       -1, -1,  6, -1, -1, -1, -1, -1,  0, -1,  7,  8, -1,  0,  9, -1, -1,
       10, -1, -1, 11, -1,  7, 12, -1,  0, -1, 13, -1, 11, -1,  0, -1, -1,
       -1, 14, 15, -1, -1, -1, -1, -1,  0, 16, 11, 17, 18, -1, 19, 16, 20,
       11, 21, 22, -1, -1, -1, -1, 23, -1, 16, -1, 24, -1, -1, 13, -1, -1,
       25, 13, 11, -1, 26, -1, -1, -1, 21, 27, -1, 28, -1, -1, -1])

## Информация о кластерах

In [120]:
newsLabels = zip_news(news, labels)
newsLabels = sorted(newsLabels, key=lambda n: n.label)
for label, group in groupby(newsLabels, lambda n: n.label):
    groupList = list(group)
    print("Cluster: %s, count news: %s, titles:" % (label, len(groupList)))
    for gr in groupList:
        print("\t" + gr.title)

Cluster: -1, count news: 578, titles:
	На Алтае заработала первая в РФ система контроля космического пространства
	Bild не публикует результаты России в медальном зачёте Олимпиады
	Глава ЛНР связал покушение на свою жизнь с украинскими спецслужбами
	Мэра города в США арестовали за попытку расплатиться наркотиками за секс с мужчинами
	В Глазго, предположительно из-за взрыва, обрушилась часть здания
	Председатель Совмина ЛНР: Состояние Плотницкого после покушения стабильное
	Легкомоторный самолёт упал в воды Ла-Манша на юго-востоке Англии
	На блошином рынке во Франции утраченную гравюру Дюрера продали за несколько евро
	Как мировые лидеры посещали Олимпиады
	Полиция отпустила афганского беженца, подозревавшегося в подготовке теракта в Париже
	Активисты оспорят в суде планы переноса советских памятников в Польше
	В московском метро снова раздают воду из-за жары
	Подозреваемый в нападении на туристов предстал перед судом Лондона
	В Домодедово досрочно приземлился самолёт из-за больного реб